<a href="https://colab.research.google.com/github/eoinmooremath/fantasy-sports/blob/main/Fantasy_Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **DraftKings Football Bet Optimizer**

*The code in this notebook can be used to create draft teams with optimal expected value in Draftkings Football Classic Mode.*

DraftKings is an online gambling websites where players makes bets on sports games. There are a variety of types of betting one can do, but the main one we will focus on is drafting football in classic mode.

To play a DraftKings football classic contest, you first select an eligable roster of available football players who are playing in real life games on a given night. This will be your team for the contest. Based on game actions those players perform, that player will get *fantasy points*. For example, if a player passes a touchdown, that player will get 4 points. If a player throws an interception, they will lose one point. If a player has a rushing touchdown, they will get 6 points. DraftKings tallys each player's point total during the game. Add up all the points of all your team's players to get your score. You compete in a pool of other teams in the contest. Prizes are awarded based on how well many points you got versus your competitors. In the type I like to play -- *50-50* -- you bet, say, 10 dollars, and the top 50% of players will get 1.8x return on their money, in this case winning 18 dollars.

The skill of DraftKings lies in your ability to draft a team. You are allotted 50,000 dollars in salary to buy the salaries of your players. Different players have different salaries. Better players have higher salaries. The team you make is independent of the teams other players make, so you can be up against teams with overlapping players. All that is required is that your total salary not exceed 50,000 dollars.

Now, this would be a very good opportunity to do deep learning and predict players point scores on a given night based upon factors such as their opposing team's lineup, weather, etc. For this first pass at the project, we are going to take a simpler approach, but one that will allow us to investigate python frameworks for parallel computing and speeding up code with just in time compilation.

DraftKings is kind enough to provide us a metric for each player called *fantasy points per game* (fppg). This is the average of fantasy points that player has scored per game during this season. We will use that as a reasonable expectation for how a player will do. Then, why not do the obvious thing -- why not just add up all the players of each position with the highest fppg?  The reason is, if you do that, you will invariably exceed 50,000 dollars salary cap.


What we would like to do, then, is pack the 50,000 dollars with the highest fppg expected value (EV) as we can.  Maybe the highest team EV comes from picking almost all of the highest EV players, and one or two low-EV players. Or maybe it comes from picking players who all have modestly high EV. It is hard for us to guess, so instead we will have a computer comb through vast numbers of possibile team combinations, compute each expected fppg, and return the best choice.



## Import the necessary libraries.  
Pandas is for handling DraftKings' .csv files of player data. Numba is for creating just-in-time code. Dask is for parallelizing the code.

In [ ]:
import pandas as pd
import numpy as np
import math
from numba import njit, prange
import dask
import itertools
from dask import delayed
import dask.config
import time

## Import and process the data.
Go to the classic NFL contest that you are interested in creating a team for. Click \"Export to CSV \" to get the team data. I recommend saving it with the link to the draft in mind. So for example, if the draft is at https://www.draftkings.com/draft/contest/167663950, I will name my file, Football_classic_167663950.csv.

Next, we will convert the .csv file into a Pandas database. We will only keep the columns we care about -- \'Name\', \'Position\', \'Salary\', and \'AvgPointsPerGame\' which we will rename \'Fppg\'. We will have one database `db_all` to hold all our players. We will have one database for each team position as well.

*About team formation --*
On DraftKings your team consists of exactly nine \"players\":  

1 quarterback; 2 runningbacks; 3 wide receivers; 1 tight end; 1 flex spot -- which is a player from among runningbacks, wide receivers, and tightends; and one defense / special teams slot. (DraftKings doesn't let you individually draft defensive / special teams players. Instead, their collective *team* scores are aggregated as \"DST\" position. So a DST *player* is actually a team.)   


In [ ]:
url = 'https://raw.githubusercontent.com/eoinmooremath/fantasy-sports/b0e5e66762b9e8f55a4ece0f755e7ef64cc793de/Football_classic_167663950.csv'
db_all = pd.read_csv(url)
db_all.rename(columns={'AvgPointsPerGame':'Fppg'}, inplace=True)
db_all = db_all.loc[:,['Name','Position','Salary','Fppg']]
db_all['Value']=db_all['Fppg']/db_all['Salary']*10000
# "Value" is the same as "expected value." The 10000 doesn't mean anything. I just like to multiply by 10000 to make the numbers easier to read.

db_d = db_all[db_all['Position']=='DST']
db_f = db_all[(db_all['Position']=='RB') | (db_all['Position']=='WR') | (db_all['Position']=='TE') ]
db_q = db_all[db_all['Position']=='QB']
db_r = db_all[db_all['Position']=='RB']
db_t = db_all[db_all['Position']=='TE']
db_w = db_all[db_all['Position']=='WR']

## Prepare the teams

Examining the data, we see there are 24 DST players, 499 flex players, 79 quarterbacks, 134 running backs, 138 tight ends, and 277 wide receivers.  That means there are `24*79*134*(134-1)*138*277*(277-1)*(277-2)*(499-6)= 48332840921317814400` possible teams. That's over 48 quintillion teams!  We don't even have time to check *one* quintillion teams for Sunday's contest, so we need to be judicious in the teams we compute the fppg for. If we have fewer players to check, there are fewer teams to check.

The way we will trim our player selection is with the `get_players_at_rate()` function below. We want to select only those players who have a good expected value compared to other players. `get_players_at_rate(z,players)` returns those players in the `players` database who have a z-score at least `z`. That means they are `z`-standard deviations above the mean. For example, `get_players_at_rate(2,db_q)` will return those quarterbacks who are at least two standard deviations above the mean of quarterbacks.  

Also, thinking about our code, we want it to be as lightweight as possible. We don't need to return the overhead of a pandas database, when we are planning on accessing this information billions of times. Instead, we will store only the necessary information in a numpy array. Each row will correspond to a player. The first column will be their index (to refer to them later in the `db_all` database once we get a team), and the last two will be their pppg rate and their salary.

After we create a roster of players to choose from, we would like to know how many possible teams we have to search through. This is what the `get_size` function is for.  For example, `get_size(db_d,db_f,db_q,db_r,db_t,db_w) = 48332840921317814400` If it is too large, we can adjust the teams with different rates, and try again.


In [ ]:
def get_players_at_value(z,players):
    # input: players == pandas array of players of a certain position, corresponding to a position.  z == desired z-score
    # special case, let z<0 to return players who have a positive rate (not necessarily above mean)
    # output: numpy array of those players with z-score at least z, in a numpy array.
    # the first column is the index. the second column is  Pppg. The third is Salary
    if z<0:
        players=players[players['Value']>0]
    if z>=0:
        players=players[players['Value']>= (players['Value'].mean()+z*players['Value'].std())]
    players = players.sort_values(by='Value', ascending=False)
    players = players.loc[:,['Fppg','Salary']]
    players = players.reset_index()
    players = players.to_numpy()

    return players

def get_size(d,f,q,r,t,w):
    num_teams = len(q)* ( len(r)* (len(r)-1) ) * ( len(w) * (len(w)-1) * (len(w)-2) ) * len(t) * len(f) * len(d)
    return num_teams


## Compute the best team

I will now give you the code which you can call to create the best team. Later I will talk about a possible work flow.

Essentially, this code is one massive for-loop. We use nested for-loops to create all possible teams among our selected players. We calculate the team's expected value by adding up the EVs of all of its players. We keep only the team with the highest EV that also has a total price less than \$50,000.

I first wrote my code naively without any optimization. It took a long time to run. I could trim the team sizes or expect long wait times. I researched a bit and found that *just in time* compilation with numba could significantly speed up my program. Numba takes the easy-to-type but slow-to-run python code and translates it into optimized machine code at runtime. I edited my program so that it would be compatible with Numba. The results were dramatic. In experiments, I saw a *speed up of around* **80x**.

After further consideration, I realized that the computations within the for-loop are independent of each other, making them well-suited for parallelization, which can further speed up the code. Dask is a parallel computing framework that allows Python code to be executed across multiple cores of your CPU or even distributed across a cluster. I generate batches of teams. Dask distributes those batches to available CPU cores, processing them concurrently. Once the computations are complete, the results are collected, and the best team among all is kept. After implementing Dash, this time *I saw another speed up around* **1.3x**  from just Numba alone. Dash with Numba provided a *total speed up of around* **100x** compared to the plain vanilla version.  

Here are all three versions, ordered from fastest to slowest.

### Dask and Numba version


In [ ]:
@njit
def inner_loop_dask(players_d, players_f, players_q, players_r, players_t, players_w, w1, w2, w3):
    # How you break this up seems arbitrary, and open to experimentation. Batches will differ according to
    # their wide receivers. So take those as constants. Inside this inner loop, we will loop over all possible
    # remaining players. You could choose to break the loop up differently, and perhaps achieve different speed-ups.
    d_len = len(players_d)
    f_len = len(players_f)
    q_len = len(players_q)
    r_len = len(players_r)
    t_len = len(players_t)
    best_EV = 0

    def test_no_repeats(arr):  # this is to make sure that we don't accidently pick the same player twice
        for x in range(len(arr)):
            for y in range(x+1,len(arr)):
                if arr[x]==arr[y]:
                    return False
        return True

    for d in range(d_len):  # the nested for-loops get the team.
        for f in range(f_len):
            for q in range(q_len):
                for r1 in range(r_len):
                    for r2 in range(r1+1, r_len):
                        for t in range(t_len):  # if the team is under $50,000 and is better than the previous best team, keep it, and its EV.
                            points_EV = (players_d[d, 1] + players_f[f, 1] +
                                            players_q[q, 1] + players_r[r1, 1] +
                                            players_r[r2, 1] + players_t[t, 1] +
                                            players_w[w1, 1] + players_w[w2, 1] +
                                            players_w[w3, 1])
                            team_price = (players_d[d, 2] + players_f[f, 2] + players_q[q, 2]
                                           +players_r[r1, 2] + players_r[r2, 2]
                                            + players_t[t, 2] +
                                            players_w[w1, 2] + players_w[w2, 2] +players_w[w3,2])
                            test_set = np.array([players_r[r1, 0], players_r[r2, 0],
                                                players_w[w1, 0], players_w[w2, 0],
                                                players_w[w3, 0], players_f[f,0]])
                            no_repeats = test_no_repeats(test_set)

                            if ((points_EV > best_EV) and (team_price < 50000) and no_repeats):
                                best_EV = points_EV

                                best_team = np.array([
                                     players_d[d, 0], players_f[f, 0], players_q[q, 0],
                                     players_r[r1, 0], players_r[r2, 0], players_t[t, 0],
                                     players_w[w1, 0], players_w[w2, 0], players_w[w3, 0]
                                ])

    return best_team, best_EV

@dask.delayed
def make_team_dask(players_d, players_f, players_q, players_r, players_t, players_w):
    w = list(itertools.combinations(range(len(players_w)), 3))
    # itertools.combinations is an efficient and fast way to generate all possible
    # 3-player permutations of wide receivers, which is what we need.  Loop over them.
    best_EV = 0
    for idx in range(len(w)):
        w1, w2, w3 = w[idx]
        team, EV = inner_loop_dask(players_d, players_f, players_q, players_r, players_t, players_w, w1, w2, w3)
        if EV >= best_EV:
            best_EV = EV
            best_team = team
    return best_team


### Numba version

In [ ]:
@njit
def make_team_njit(players_d, players_f, players_q, players_r, players_t, players_w):
    d_len = len(players_d)
    f_len = len(players_f)
    q_len = len(players_q)
    r_len = len(players_r)
    t_len = len(players_t)
    w_len = len(players_w)
    best_EV = 0

    def test_no_repeats(arr):
        for x in range(len(arr)):
            for y in range(x+1,len(arr)):
                if arr[x]==arr[y]:
                    return False
        return True

    for d in range(d_len):
        for f in range(f_len):
            for q in range(q_len):
                for r1 in range(r_len):
                    for r2 in range(r1+1, r_len):
                        for t in range(t_len):
                            for w1 in range(w_len):
                                for w2 in range(w1+1, w_len):
                                    for w3 in range(w2, w_len):
                                        points_EV = (players_d[d, 1] + players_f[f, 1] +
                                                        players_q[q, 1] + players_r[r1, 1] +
                                                        players_r[r2, 1] + players_t[t, 1] +
                                                        players_w[w1, 1] + players_w[w2, 1] +
                                                        players_w[w3, 1])
                                        team_price = (players_d[d, 2] + players_f[f, 2] + players_q[q, 2]
                                                      +players_r[r1, 2] + players_r[r2, 2]
                                                        + players_t[t, 2] +
                                                        players_w[w1, 2] + players_w[w2, 2] +players_w[w3,2])
                                        test_set = np.array([players_r[r1, 0], players_r[r2, 0],
                                                            players_w[w1, 0], players_w[w2, 0],
                                                            players_w[w3, 0], players_f[f,0]])
                                        no_repeats = test_no_repeats(test_set)

                                        if ((points_EV > best_EV) and (team_price < 50000) and no_repeats):
                                            best_EV = points_EV

                                            best_team = np.array([
                                                players_d[d, 0], players_f[f, 0], players_q[q, 0],
                                                players_r[r1, 0], players_r[r2, 0], players_t[t, 0],
                                                players_w[w1, 0], players_w[w2, 0], players_w[w3, 0]
                                            ])
    return best_team




### Vanilla version (no optimization)

In [ ]:
def make_team_vanilla(players_d, players_f, players_q, players_r, players_t, players_w):  #this is the same as the numba version, just without the decorator
    d_len = len(players_d)
    f_len = len(players_f)
    q_len = len(players_q)
    r_len = len(players_r)
    t_len = len(players_t)
    w_len = len(players_w)
    best_EV = 0

    def test_no_repeats(arr):
        for x in range(len(arr)):
            for y in range(x+1,len(arr)):
                if arr[x]==arr[y]:
                    return False
        return True

    for d in range(d_len):
        for f in range(f_len):
            for q in range(q_len):
                for r1 in range(r_len):
                    for r2 in range(r1+1, r_len):
                        for t in range(t_len):
                            for w1 in range(w_len):
                                for w2 in range(w1+1, w_len):
                                    for w3 in range(w2, w_len):
                                        points_EV = (players_d[d, 1] + players_f[f, 1] +
                                                        players_q[q, 1] + players_r[r1, 1] +
                                                        players_r[r2, 1] + players_t[t, 1] +
                                                        players_w[w1, 1] + players_w[w2, 1] +
                                                        players_w[w3, 1])
                                        team_price = (players_d[d, 2] + players_f[f, 2] + players_q[q, 2]
                                                      +players_r[r1, 2] + players_r[r2, 2]
                                                        + players_t[t, 2] +
                                                        players_w[w1, 2] + players_w[w2, 2] +players_w[w3,2])
                                        test_set = np.array([players_r[r1, 0], players_r[r2, 0],
                                                            players_w[w1, 0], players_w[w2, 0],
                                                            players_w[w3, 0], players_f[f,0]])
                                        no_repeats = test_no_repeats(test_set)

                                        if ((points_EV > best_EV) and (team_price < 50000) and no_repeats):
                                            best_EV = points_EV

                                            best_team = np.array([
                                                players_d[d, 0], players_f[f, 0], players_q[q, 0],
                                                players_r[r1, 0], players_r[r2, 0], players_t[t, 0],
                                                players_w[w1, 0], players_w[w2, 0], players_w[w3, 0]
                                            ])
    return best_team




## Workflow

The calculations performed inside the for-loops are relatively simple. Theoretically, the time it takes to run the program should scale linearly with the number of teams compared.  I find this to hold experimentally, as well. This suggests a workflow where we can target the program to run in a desired amount of time.

Create a small roster using `get_players_at_rate`. Calculate the number of possible teams it generates using the `get_size` function, storing the result in a variable `num_teams`. Next, measure the time it takes to compute the best team from that roster and call it, say, `t`. Let `t_max` be the amount of time you are willing to spend on the computation.

Then, you are looking for a roster size equal to about `num_teams * (t_max / t)`.

Play around with the `get_players_at_rate` function to produce rosters until you hit your goal. Note, you might not want to get all positions at the same right. If you do, you might be left with a roster with lots of flex-spots, but few DST choices.

This isn't *quite* right, especially for Dask, as the parallelization adds overhead that will disproportionately impact smaller runs. This is why you will see the Numba code runs the fastest below.  

Note also that we run the Dask code a little differently than usual.

In [ ]:
strict_rate = 2.5

select_d = get_players_at_value(1,db_d)
select_f = get_players_at_value(strict_rate,db_f)
select_q = get_players_at_value(2,db_q)
select_r = get_players_at_value(strict_rate,db_r)
select_t = get_players_at_value(strict_rate,db_t)
select_w = get_players_at_value(strict_rate,db_w)

num_teams = get_size(select_d,select_f,select_q,select_r,select_t,select_w)
print(f"Number of possible teams: {num_teams}")
print()


team_dask_ddo = make_team_dask(select_d,select_f,select_q,select_r,select_t,select_w)
#this makes a 'Dask delayed object' only. Compute the result by running .compute() on it.

start_time = time.perf_counter()
team_dask=team_dask_ddo.compute()
end_time = time.perf_counter()
t_dask = end_time-start_time
print(f"Dask execution time: {t_dask} seconds")

start_time = time.perf_counter()
team_njit = make_team_njit(select_d,select_f,select_q,select_r,select_t,select_w)
end_time = time.perf_counter()
t_njit = end_time-start_time
print(f"Numba execution time: {t_njit} seconds")

start_time = time.perf_counter()
team_van = make_team_vanilla(select_d,select_f,select_q,select_r,select_t,select_w)
end_time = time.perf_counter()
t_van = end_time-start_time
print(f"Unoptomized execution time: {t_van} seconds")

print()

print(f"For Dask, the salary is {db_all.loc[team_dask,:]['Salary'].sum()} and points EV is  {db_all.loc[team_dask,:]['Fppg'].sum()}")
print(db_all.iloc[team_dask,:])
print()

print(f"For Numba, the salary is {db_all.loc[team_njit,:]['Salary'].sum()} and points EV is  {db_all.loc[team_njit,:]['Fppg'].sum()}")
print(db_all.iloc[team_njit,:])
print()

print(f"For Vanilla, the salary is {db_all.loc[team_van,:]['Salary'].sum()} and points EV is  {db_all.loc[team_van,:]['Fppg'].sum()}")
print(db_all.iloc[team_van,:])
print()
print("It's a good check to see that all give the same result!")

In [ ]:
strict_rate = 2

select_d = get_players_at_value(1,db_d)
select_f = get_players_at_value(strict_rate,db_f)
select_q = get_players_at_value(strict_rate,db_q)
select_r = get_players_at_value(strict_rate,db_r)
select_t = get_players_at_value(strict_rate,db_t)
select_w = get_players_at_value(strict_rate,db_w)
num_teams2 = get_size(select_d,select_f,select_q,select_r,select_t,select_w)
print(f"Number of possible teams: {num_teams2}")
print(f"Expect times to take  {num_teams2/num_teams} x longer.")

Next we will test slightly larger arrays, especially to see how Dask does when the overhead will be relatively smaller compared to the whole task. We anticipate that running the unoptimized version will take a while, so we will run that in a separate cell.

In [ ]:
team_dask_ddo2 = make_team_dask(select_d,select_f,select_q,select_r,select_t,select_w)

start_time = time.perf_counter()
team_dask2=team_dask_ddo2.compute()
end_time = time.perf_counter()
t_dask2 = end_time-start_time
print(f"Dask execution time: {t_dask2} seconds. That is {t_dask2/t_dask} slower than the previous run. We expected a slowdown factor of {num_teams2/num_teams}.")

start_time = time.perf_counter()
team_njit2 = make_team_njit(select_d,select_f,select_q,select_r,select_t,select_w)
end_time = time.perf_counter()
t_njit2 = end_time-start_time
print(f"Numba execution time: {t_njit2} seconds.  That is {t_njit2/t_njit} slower than the previous run. We expected a slowdown factor of {num_teams2/num_teams}.")
print()
print(f"The salary is {db_all.loc[team_dask2,:]['Salary'].sum()} and points EV is  {db_all.loc[team_dask2,:]['Fppg'].sum()}")
print(db_all.iloc[team_dask2,:])
print()
print('Compare this to the team we got before:')
print(f"The salary is {db_all.loc[team_van,:]['Salary'].sum()} and points EV is  {db_all.loc[team_van,:]['Fppg'].sum()}")
print(db_all.iloc[team_van,:])

As we see, Dash slowed down less than expected, which means that it sped up *more* than expected! This is for the reasons of overhead explained earlier. We also see that both rosters produce the same team. That's a little dissapointing. Hopefully we will get a better team when we compute over an even larger roster.

You can try running the code below to see how the unoptimized version does, however you might run into problems if you are using the free version of Google Colab. Mine timed out after many hours running on Google Colab.

However, I did the same experiments on my personal computer, which is a little faster than my Google Colab instance. It took 184.86 seconds to run.  There, (and this also was about the same on larger runs) --  

 **Dask with Numba was 1.31x faster than just Numba and 104x faster than unoptimized code. Numba was 79.6x faster than unoptimized code.**



In [ ]:
start_time = time.perf_counter()
team_van2 = make_team_vanilla(select_d,select_f,select_q,select_r,select_t,select_w)
end_time = time.perf_counter()
t_van2 = end_time-start_timeprint(f"Numba execution time: {t_van2} seconds.  That is {t_van2/t_van} slower than the previous run. We expected a slowdown factor of {num_teams2/num_teams}.")
print()
print(f"Dask with Numba was {t_njit2/t_dask2 }x faster than just Numba and {t_van2/t_dask2}x faster than unoptimized code. Numba was {t_van2/t_njit2}x faster than unoptimized code.")

Now we will compute an actual, realistic roster, this time with just Dask and Numba.

In [ ]:
rate = 1

select_d = get_players_at_value(-1,db_d)
select_f = get_players_at_value(2,db_f)
select_q = get_players_at_value(rate,db_q)
select_r = get_players_at_value(rate,db_r)
select_t = get_players_at_value(rate,db_t)
select_w = get_players_at_value(1.5,db_w)
num_teams3 = get_size(select_d,select_f,select_q,select_r,select_t,select_w)
print(f"Number of possible teams: {num_teams2}")
print(f"Expect it to take {num_teams3/num_teams2*t_dask2} seconds or {num_teams3/num_teams2*t_dask2/3600} hours.")



In [ ]:
team_dask_ddo3 = make_team_dask(select_d,select_f,select_q,select_r,select_t,select_w)
start_time = time.perf_counter()
team_dask3=team_dask_ddo3.compute()
end_time = time.perf_counter()


print(f"That took {end_time-start_time} seconds. We expected it to take {num_teams3/num_teams2*t_dask2} seconds .")
print()
print(f"The salary is {db_all.loc[team_dask3,:]['Salary'].sum()} and points EV is  {db_all.loc[team_dask3,:]['Fppg'].sum()}")
print(db_all.iloc[team_dask3,:])
print()
print('Compare this to the team we got before:')
print(f"The salary is {db_all.loc[team_van,:]['Salary'].sum()} and points EV is  {db_all.loc[team_van,:]['Fppg'].sum()}")
print(db_all.iloc[team_van,:])

## Future directions

The next logical thing to do would be to have the algortihm run in parallel on GPUs or clusters of GPUs. I looked into doing this, but haven't yet figured out how to do it.

More impactfully, I would like to apply deep learning to get more accurate points forecast.  Currently I use a player's average pppg to estimate how many player points they will score in their game. There are certain flaws with this thinking, though. For example, if my quarterback is playing against my defensive team, do I expect them both to have good (or average) games?

Using deep learning, we could train a model on historical NFL and college football game data, stripping the data that would identify specific players, and instead focus on how different positions correlate with other positions in generating points. Later, fine-tune the model on today's players, capturing specific player-player interactions.

We could also use machine learning to return not just the highest EV team, but also a number of high EV teams which cummulatively have, say, the lowest risk. We would create a portfolio of teams to bet on.

With more accurate point forecast, and more robust team creation, we would then compute the best teams using GPU accelerated technology.

That's the vision for where I want to take this project. If you have any ideas or want to help, please let me know.

This code can easily be adapted to other sports on DraftKings, and I have already done so for golf.